In [ ]:
from utils.preprocessor import load_datasets
vocabularies = ['AODN Discovery Parameter Vocabulary']
targetDS, keywordDS, K = load_datasets()

In [ ]:
K

In [ ]:
# targetDS is the subset of dataset in which keywords are empty, used as the target dataset for predicsion
targetDS.info()

In [ ]:
# keywordsDS is the subset of dataset in which the records has AODN vocabulary keywords, used as the dataset for traning and testing a ML model
keywordDS.describe()

In [ ]:
# check keywords in sample dataset - no empty keywords in sample dataset
list_lengths = keywordDS['keywords'].apply(len)
empty_keywords_records_index= list_lengths[list_lengths == 0].index.tolist()
empty_keywords_records_index

Have a deeper look of these records, we found that they have the AODN Discovery Parameter vocabulary field but have no keywords, so we removed these records. (see example [4673208b-def5-4340-9818-8419496e4863](https://geonetwork-edge.edge.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/4673208b-def5-4340-9818-8419496e4863), [f55a53db-09fc-480d-aa9e-2aa6bb304b8c](f55a53db-09fc-480d-aa9e-2aa6bb304b8c), and [d265307c-5a6a-4a52-b352-35ad904fca52](https://geonetwork-edge.edge.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/d265307c-5a6a-4a52-b352-35ad904fca52)).

In [ ]:
empty_keywords_records = []
for index in empty_keywords_records_index:
    empty_keywords_records.append(keywordDS.iloc[index]['id'])
empty_keywords_records
keywordDS_dropped = keywordDS[~keywordDS['id'].isin(empty_keywords_records)]
keywordDS_dropped.info()

In [ ]:
# K is the keywords matrix
from matplotlib import pyplot as plt

category_distribution = K.copy()
category_distribution = category_distribution.sum()

plt.figure(figsize=(15,50))
category_distribution.sort_values().plot(kind='barh', color='skyblue', edgecolor='black')
plt.title("Keywords Distribution")
plt.ylabel("Keywords")
plt.xlabel("Count of Related Metadata Records")
plt.xticks(fontsize=12)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

We can find some catogories only has one related records. This means within the particular label, there is only one positive sample, and others are all negative. In this case, we simply duplicate the positive samples to inrease the volumn of positive samples on these labels.

In [7]:
category_distribution.sort_values()
category_distribution_df = category_distribution.to_frame(name='count')
rare_category = category_distribution_df[category_distribution_df['count']==1] + category_distribution_df[category_distribution_df['count']==2]
rare_category.to_csv('./output/droped_keywords.csv')

In [ ]:
count_K = K.copy()
count_K['Label Count'] = K.sum(axis=1)

print(f"Average number of labels each record has: {count_K['Label Count'].mean()}")
print(f"Maximum number of labels a record has: {count_K['Label Count'].max()}")
print(f"Minium number of labels a record has: {count_K['Label Count'].min()}")

# count_keywordsDS = keywordDS.copy()
# count_keywordsDS['keywords count'] = count_keywordsDS['keywords'].apply(len)
# count_keywordsDS.loc[count_keywordsDS['keywords count'] == 66].shape[0]


Based on our statistical analysis, we identified several key challenges in this multi-label classification task:

- **Global Label Imbalance**: The distribution of keywords across the dataset is highly imbalanced. While a few metadata records are associated with a large number of keywords (up to 66), many records are linked with only a small number of keywords (as few as 1). This significant variation poses difficulties for the model to accurately learn and predict the diverse label distributions.

- **Internal Label Imbalance**: Within each metadata record, there is also a strong imbalance between positive and negative labels. Despite having a total of 392 unique labels, each record is typically annotated with only a small subset of these labels, averaging around 4.14 keywords per record. This means that for any given record, the number of positive labels is relatively small compared to the number of negative labels, which can hinder the model's ability to effectively identify less frequently occurring keywords.

***Imbalance Ratio***

To make this clear, we introduce two metrics for demonstrating the above imbalance issues:
- *global imbalance ratio*:
- *internal imbalance ratio*:

## Prepare Train and Test Sets
We select description embedding as input X, and keyword vetors as output Y. We split the train, test and vailidation sets follow the propotion of 80%-10%-10%.

In [ ]:
from utils.preprocessor import prepare_train_validation_test
import numpy as np

X = np.array(keywordDS['embedding'].tolist())
dimension, n_labels, X_train, Y_train, X_test, Y_test = prepare_train_validation_test(X, K)
print(Y_train.shape)

Keep the aforementioned imbalanced label issue in mind, we first address challenge 1 - imbalance label issue at a global view. We compute the weight for each class (keyword) by $\omega_k = \frac{1}{\sum_1^m h(m)}$, where $h(m)=1$ if a metadata record $m$ has label $k$, otherwise $h(m)=0$.

In [ ]:
from utils.preprocessor import resampling
resampling(X_train, Y_train)

In [31]:
label_frequency = np.sum(Y_train, axis=0)
total_samples = Y_train.shape[0]
# alpha = 10
# label_weights = (1 - (label_frequency / total_samples)) ** alpha
epsilon = 1e-6
label_weights = np.minimum(1, 1 / (label_frequency + epsilon))

label_weight_dict = {i: label_weights[i] for i in range(len(label_weights))}

## Training Model
We apply the KNN model for this task, and set $k=5$.

In [ ]:
import model.keywordModel as km

model, history = km.keyword_model(X_train,Y_train, X_test, Y_test, label_weight_dict, dimension, n_labels)

## Evaluation
Evaluate the trained model by adapting evaluation metrics:
- Precision
- Recall
- F1 Score
- Hamming Loss
- ROC

In [ ]:
from model.keywordModel import evaluation

predictions = model.predict(X_test)
confidence = 0.7
predicted_labels = (predictions > confidence).astype(int)
eval = evaluation(Y_test=Y_test, predictions=predicted_labels)
print(eval)

In [14]:
from model.keywordModel import prediction
prediction(targetDS, model, confidence=0.5, labels=K.columns)

## Parameter Setting
|parameter|value|definition|
| ---- | ---- | ---- |
| epoch | 100 | number of epochs for training a model (note: early stop used)
| batch_size | 32 | the size of a batch |
| patience of EarlyStopping | 3 | ---- |
| patience of ReduceLROnPlateau | 5 | ---- |
| validation_split | 0.1 | percentage to split the train set with a validation set |
| confidencce | 0.5 | the threshold to predict a label as positive |
| n_split for MultilabelStratifiedShuffleSplit | 5 | rounds of cross validation | 
| test_size for MultilabelStratifiedShuffleSplit | 0.2 | percentage to split test set |




1. K (before resampling)

|k|Precision|F1 Score|Recall|Hamming Loss|Accuracy|
|----|----|----|----|----|----|
| 10 | 0.8541176470588235 | 0.6570135746606335 | 0.5338235294117647 | 0.006080734180464639 | 0.5408805031446541 |
| 8 | 0.8492990654205608 | 0.6561371841155235 | 0.5345588235294118 | 0.00611282248748556 | 0.5471698113207547 |
| 5 | 0.8340659340659341 | 0.6687224669603524 | 0.5580882352941177 | 0.0060326017199332565 | 0.5628930817610063 |
| 3 | 0.75 | 0.6296928327645052 | 0.5426470588235294 | 0.006963162623539982 | 0.559748427672956 |
| 1 | 0.48342541436464087 | 0.4985754985754986 | 0.5147058823529411 | 0.011295084071364396 | 0.559748427672956 |

Comparing the result, we select $K=5$ as it has a better performance.

2. Resample strategy

| strategy |Precision|F1 Score|Recall|Hamming Loss|Accuracy|
|----|----|----|----|----|----|
| RUS (majority) | 0.7361601884570083 | 0.5658669081032142 | 0.45955882352941174  | 0.0076931716082659475 | 0.2358490566037736 |
| ROS (minority) | 0.8340659340659341 | 0.6687224669603524 | 0.5580882352941177| 0.0060326017199332565 | 0.5628930817610063 |
| SMOTE |----|----|----|----|----|


## Comparison
| method | Precision | Recall | F1 Core | Hamming Loss|
|----|----|----|----|----|
| MLP | 0.8506 | 0.5316 | 0.6543 | 0.0061 |
| MLP (SS)| 0.8605 | 0.5787 | 0.6920 | 0.0052 |
| MLP (SS + ROS) | 0.8744 | 0.5644 | 0.6860 | 0.0052 |
| MLP (SS + RUS) | 0.7337 | 0.2321 | 0.3527 | 0.0087 |
| DT | 0.5988 | 0.4971 | 0.5432 | 0.0091 |
| DT (SS)| 0.6000 | 0.5246 | 0.5598 | 0.0084 |
| DT (SS + ROS)| 0.5964 | 0.5262 | 0.5591 | 0.0084 |
| DT (SS + RUS)| 0.5100 | 0.3633 | 0.4243 | 0.0100 |
| KNN | 0.8341 | 0.5581 | 0.6687 | 0.0060 |
| KNN (SS)| 0.8521 | 0.6089 | 0.7102 | 0.0050 |
| KNN (SS + ROS) | 0.8521 | 0.6089 | 0.7102 | 0.0050 |
| KNN (SS + RUS) | 0.8247 | 0.5199 | 0.6377 | 0.0060 |

Conclusion:
1. Using Stratified Sampling for splitting the train and test sets improves model performance across all three models.
2. Adopt oversampling methods in the resampling process, as ROS (Random Oversampling) performs better than RUS (Random Undersampling) in all three models.

Models
- MLP: Our trained MultiLayer Perceptron model
- DT: DecisionTreeClassifier (baseline model)
- KNN: KNeighborsClassifier (baseline model)

Samplying/Resampling Strategy:
- SS: Stratified Sampling. Stratifying the data when preparing train and test sets ensures that the distributions of positive and negative samples align with the original dataset.
- ROS: Random Oversampling for the train set, used to increase the number of minority class samples.
- RUS: Random Undersampling for the train set, used to decrease the number of majority class samples.



Other things I will investigate next are:
1. Considering whether class weight in the loss function truly impacts model performance. As shown below, the loss function without class weights performs better. I will need to explore how to utilize this relationship when training the model.

| method | Precision | Recall | F1 Core | Hamming Loss|
|----|----|----|----|----|
| MLP | 0.8506 | 0.5316 | 0.6543 | 0.0061 |
| MLP (weighted) | 0.7554 | 0.3066 | 0.4362 | 0.0086 |

2. Try other resampling strategy like [SMOTE](https://arxiv.org/abs/1106.1813).
3. Keeping the adaptability of our model in mind: considering additional vocabularies as needed.




### Update

do foca loss function:
| method | Precision | Recall | F1 Core | Hamming Loss|
|----|----|----|----|----|
| MLP (binary cross entropy)| 0.8072 | 0.5974 | 0.6866 | 0.0056 |
| MLP (foca loss(gamma=2., alpha=0.25))| 0.8740 | 0.4451 | 0.5898 | 0.0063 |
| MLP + ROS | 0.6038 | 0.6672 | 0.6339 | 0.0079 |
| MLP + ROS + Foca Loss | 0.6213 | 0.4250 | 0.5048 | 0.0085 |
| ROS + MLP | 0.8596 | 0.8446 | 0.8521 | 0.0056 |
| ROS + MLP + Foca Loss| 0.9459 | 0.7283 | 0.8229 | 0.0060 |
| ROS + MLP + ROS|  0.9239 | 0.8645 | 0.8932 | 0.0040 | 
| ROS + MLP + ROS + Foca Loss | 0.9740 | 0.7931 | 0.8743 | 0.0044 |

You are a marine science metadata officer. I will give 5 items one by one. First refomat the given text to description and keywords, don't provide keywords by yourself. Then score how the keywords maps the item (range from 0-10). In the end, return the average score of all 5 items, and also the min and max scores. 

The score rule is applied for each keyword-item description mapping as follows:

All four metrics will now be scored on a scale of 0-10, and the total score will be the sum of the four metrics, ranging from 0-40.

1. Relevance (0-10 points)
10/10: The keyword is entirely relevant, capturing the primary focus of the description perfectly.
7-9/10: The keyword is relevant but does not fully encompass all important aspects of the description.
4-6/10: The keyword is somewhat relevant but misses significant portions of the description's focus.
1-3/10: The keyword is largely irrelevant, capturing very little of the core concept.
0/10: The keyword is entirely unrelated to the description.
2. Precision (0-10 points)
10/10: All aspects of the keyword prediction are correct, with no irrelevant or off-topic predictions.
7-9/10: The majority of the predicted keywords are correct, with only minor inaccuracies.
4-6/10: Some correct predictions, but with significant inaccuracies or irrelevant keywords.
1-3/10: Mostly inaccurate predictions, with only minor correct aspects.
0/10: Entirely incorrect or irrelevant predictions.
3. Recall (0-10 points)
10/10: The keyword prediction captures all major relevant concepts from the description.
7-9/10: Most relevant concepts are captured, but a few significant elements are missing.
4-6/10: Some relevant concepts are captured, but many important ones are missing.
1-3/10: Very few relevant concepts are captured, with most of the important ones missing.
0/10: No relevant concepts are captured at all.
4. Contextual Understanding (0-10 points)
10/10: The keyword prediction demonstrates a full understanding of the description's context, capturing subtle nuances.
7-9/10: Good understanding of the context, though with minor gaps in subtlety or depth.
4-6/10: Basic understanding of the context, but with significant gaps in deeper understanding.
1-3/10: Minimal understanding of the context, capturing only superficial aspects.
0/10: No understanding of the context, with predictions unrelated to the description.
Total Score:
The total score will be the sum of the four metrics, ranging from 0 to 40.

If there are multiple keywords in an item, the score is the averaged value of each keyword-item mapping.

Count the items I provided, and let me know if 5 items are provided so I'll end this game.

Do you understand?

5 vocabularies results:
1. loss function strategy

| model | precision | recall | F1 score | hamming loss |
|----|----|----|----|----|
|MLP (binary crossentropy)| 0.5148 | 0.4086 | 0.4556 | 0.0087 |
|MLP (weighted binary cross entropy)
|MLP (Focal Loss)|  0.7758 | 0.4272 | 0.5510 | 0.0062 |